In [ ]:
%run cliffcommutil.py
model_ws = 188065
model_suffix = ".pg.G.D.pyr"
procid = 0
mag_list = util.load("mag_list")
metabolites = util.load("metabolites")
feature_probabilities = util.load("feature_probabilities")
reaction_probabilities = util.load("reaction_probabilities"+str(procid),{})
gf_phenotype_results = util.load("new_gf_phenotype_results_"+str(procid),{})
probability_finished = util.load("probability_finished_"+str(procid),[])
problemlist = util.load("problemlist",[])
auxo_media = util.msrecon.get_media("181152/AuxoMedia")
gmm_base_media = util.msrecon.get_media("181152/BaseAerobicMM")
uptake_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="uptake"
)
excretion_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="excretion"
)
growth_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=gmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
phenosets = {"uptake":uptake_phenoset,"excretion":excretion_phenoset,"growth":growth_phenoset}
for i,item in enumerate(mag_list):
    asvname = mag_to_name[item[1]]
    if i%8 == procid:
        if asvname not in probability_finished and asvname not in problemlist:
            mdlutl = util.msrecon.get_model(asvname+model_suffix,model_ws)
            reaction_probabilities[asvname] = {}
            for rxn in mdlutl.model.reactions:
                highest_prob = None
                for gene in rxn.genes:
                    if gene.id in feature_probabilities[item[1]]:
                        if highest_prob == None or feature_probabilities[item[1]][gene.id] > highest_prob:
                            highest_prob = feature_probabilities[item[1]][gene.id]
                if highest_prob != None:
                    rxn.probability = highest_prob
                    reaction_probabilities[asvname][rxn.id] = highest_prob
            
            genome = util.msrecon.get_msgenome_from_ontology(mdlutl.model.genome_ref,native_python_api=True,output_ws=None)
            reaction_hash = genome.annoont.get_reaction_gene_hash(feature_type="gene")
            for rxn in reaction_hash:
                highest_prob = None
                for gene in reaction_hash[rxn]:
                    if gene in feature_probabilities[item[1]]:
                        if highest_prob == None or feature_probabilities[item[1]][gene] > highest_prob:
                            highest_prob = feature_probabilities[item[1]][gene]
                if highest_prob != None and (rxn+"_c0" not in reaction_probabilities[asvname] or highest_prob >= reaction_probabilities[asvname][rxn+"_c0"]):
                    if rxn+"_c0" in mdlutl.model.reactions:
                        mdlutl.model.reactions.get_by_id(rxn+"_c0").probability = highest_prob
                    reaction_probabilities[asvname][rxn+"_c0"] = highest_prob
            
            filters = mdlutl.get_attributes("gf_filter")
            tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
            msgapfill = MSGapfill(
                mdlutl,
                [util.msrecon.get_template(mdlutl.model.template_ref)],
                [],
                tests,
                blacklist=[],
                default_target="bio1",
                minimum_obj=0.01,
                base_media=None,
                base_media_target_element=None
            )

            #Adding missing transporter for metabolites to gapfilling database
            for cpd in metabolites:
                if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                    transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)

            coefficients = {}
            gf_penalties = msgapfill.gfpkgmgr.getpkg("GapfillingPkg").gapfilling_penalties
            gfrxn = 0
            probrxn = 0
            otherrxn = 0
            for reaction in msgapfill.gfmodelutl.model.reactions:
                if reaction.id in reaction_probabilities[asvname]:
                    probrxn += 2
                    coefficients[">"+reaction.id] = 1-reaction_probabilities[asvname][reaction.id]
                    coefficients["<"+reaction.id] = 1-reaction_probabilities[asvname][reaction.id]
                elif reaction.id in gf_penalties:
                    if "forward" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients[">"+reaction.id] = 1+gf_penalties[reaction.id]["forward"]
                    else:
                        otherrxn += 1
                        coefficients[">"+reaction.id] = 0.95
                    if "reverse" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients["<"+reaction.id] = 1+gf_penalties[reaction.id]["reverse"]
                    else:
                        otherrxn += 1
                        coefficients["<"+reaction.id] = 0.95
                else:
                    otherrxn += 2
                    coefficients[">"+reaction.id] = 0.95
                    coefficients["<"+reaction.id] = 0.95
            print(asvname,"GF:",gfrxn,"Prob:",probrxn,"Other:",otherrxn)

            msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True,base_filter_only=True)
            
            gf_phenotype_results[asvname] = {}
            for phenoid in phenosets:
                gf_phenotype_results[asvname][phenoid] = {}
                output = phenosets[phenoid].simulate_phenotypes(
                    msgapfill.gfmodelutl,
                    multiplier=2,
                    add_missing_exchanges=True,
                    save_fluxes=False,
                    save_reaction_list=True,
                    gapfill_negatives=False,
                    msgapfill=None,
                    test_conditions=None,
                    ignore_experimental_data=True,
                    flux_coefficients=coefficients
                )
                for index, row in output["details"].iterrows():
                    if "reactions" in output["data"][row["Phenotype"]]:
                        output["data"][row["Phenotype"]]["average_probability"] = 0
                        for rxn in output["data"][row["Phenotype"]]["reactions"]:
                            direction = rxn[0:1]
                            rxnid = rxn[1:]
                            if direction == ">":
                                if rxnid not in gf_penalties or "forward" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities[asvname]:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[asvname][rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                            elif direction == "<":
                                if rxnid not in gf_penalties or "reverse" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities[asvname]:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[asvname][rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                        output["data"][row["Phenotype"]]["average_probability"] = output["data"][row["Phenotype"]]["average_probability"]/len(output["data"][row["Phenotype"]]["reactions"])
                    gf_phenotype_results[asvname][phenoid][row["Phenotype"]] = output["data"][row["Phenotype"]]
            probability_finished.append(asvname)
            util.save("new_gf_phenotype_results_"+str(procid),gf_phenotype_results)
            util.save("reaction_probabilities"+str(procid),reaction_probabilities)
            util.save("probability_finished_"+str(procid),probability_finished)